# NLP Code Along

For this code along we will build a spam filter! We'll use the various NLP tools we learned about as well as a new classifier, Naive Bayes.

We'll use a classic dataset for this - UCI Repository SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [ ]:
data = spark.read.csv("smsspamcollection/SMSSpamCollection",inferSchema=True,sep='\t')

In [ ]:
data = data.withColumnRenamed('_c0','class').withColumnRenamed('_c1','text')

In [ ]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



## Clean and Prepare the Data

** Create a new length feature: **

In [ ]:
from pyspark.sql.functions import length

In [ ]:
data = data.withColumn('length',length(data['text']))

In [ ]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [ ]:
# Pretty Clear Difference
data.groupby('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.48663212435233|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [ ]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer

tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')
idf = IDF(inputCol="c_vec", outputCol="tf_idf")
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

In [ ]:
clean_up = VectorAssembler(inputCols=['tf_idf','length'],outputCol='features')

### The Model

We'll use Naive Bayes, but feel free to play around with this choice!

In [ ]:
from pyspark.ml.classification import NaiveBayes

In [ ]:
# Use defaults
nb = NaiveBayes()

### Pipeline

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,tokenizer,stopremove,count_vec,idf,clean_up])

In [ ]:
cleaner = data_prep_pipe.fit(data)

In [ ]:
clean_data = cleaner.transform(data)

### Training and Evaluation!

In [ ]:
clean_data = clean_data.select(['label','features'])

In [ ]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13461,[8,12,33,6...|
|  0.0|(13461,[0,26,308,...|
|  1.0|(13461,[2,14,20,3...|
|  0.0|(13461,[0,73,84,1...|
|  0.0|(13461,[36,39,140...|
|  1.0|(13461,[11,57,62,...|
|  0.0|(13461,[11,55,108...|
|  0.0|(13461,[133,195,4...|
|  1.0|(13461,[1,50,124,...|
|  1.0|(13461,[0,1,14,29...|
|  0.0|(13461,[5,19,36,4...|
|  1.0|(13461,[9,18,40,9...|
|  1.0|(13461,[14,32,50,...|
|  0.0|(13461,[42,99,101...|
|  0.0|(13461,[567,1744,...|
|  1.0|(13461,[32,113,11...|
|  0.0|(13461,[86,224,37...|
|  0.0|(13461,[0,2,52,13...|
|  0.0|(13461,[0,77,107,...|
|  1.0|(13461,[4,32,35,6...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
(training,testing) = clean_data.randomSplit([0.7,0.3])

In [ ]:
spam_predictor = nb.fit(training)

In [ ]:
data.printSchema()

root
 |-- label: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [ ]:
test_results = spam_predictor.transform(testing)

In [ ]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13461,[0,1,2,14,...|[-612.34877984332...|[0.99999999999999...|       0.0|
|  0.0|(13461,[0,1,5,15,...|[-742.97388469249...|[1.0,5.5494439698...|       0.0|
|  0.0|(13461,[0,1,6,16,...|[-1004.8197043274...|[1.0,5.0315468936...|       0.0|
|  0.0|(13461,[0,1,12,34...|[-879.22017540506...|[1.0,1.0023148863...|       0.0|
|  0.0|(13461,[0,1,15,33...|[-216.47131414494...|[1.0,1.1962236837...|       0.0|
|  0.0|(13461,[0,1,16,21...|[-673.71050817005...|[1.0,1.5549413147...|       0.0|
|  0.0|(13461,[0,1,22,26...|[-382.58333036006...|[1.0,1.7564627587...|       0.0|
|  0.0|(13461,[0,1,25,66...|[-1361.5572580867...|[1.0,2.1016772175...|       0.0|
|  0.0|(13461,[0,1,33,46...|[-378.04433557629...|[1.0,6.5844301586...|       0.0|
|  0.0|(13461,[0

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting spam was: {}".format(acc))

Accuracy of model at predicting spam was: 0.9248020435242028


Not bad considering we're using straight math on text data! Try switching out the classification models! Or even try to come up with other engineered features!

## Great Job!